<a href="https://colab.research.google.com/github/mohmaed7777/ICR-Competition/blob/main/ensembling_with_XGBoost%2C_LGBM_%26_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
!mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions download -c icr-identify-age-related-conditions

  0% 0.00/150k [00:00<?, ?B/s]
100% 150k/150k [00:00<00:00, 103MB/s]


In [6]:
! unzip icr-identify-age-related-conditions.zip

Archive:  icr-identify-age-related-conditions.zip
  inflating: greeks.csv              
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [7]:
import numpy as np
import pandas as pd
import sklearn
import random
import warnings
warnings.filterwarnings('ignore')
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('dark')

In [8]:
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')
train_data.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [9]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 58 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      617 non-null    object 
 1   AB      617 non-null    float64
 2   AF      617 non-null    float64
 3   AH      617 non-null    float64
 4   AM      617 non-null    float64
 5   AR      617 non-null    float64
 6   AX      617 non-null    float64
 7   AY      617 non-null    float64
 8   AZ      617 non-null    float64
 9   BC      617 non-null    float64
 10  BD      617 non-null    float64
 11  BN      617 non-null    float64
 12  BP      617 non-null    float64
 13  BQ      557 non-null    float64
 14  BR      617 non-null    float64
 15  BZ      617 non-null    float64
 16  CB      615 non-null    float64
 17  CC      614 non-null    float64
 18  CD      617 non-null    float64
 19  CF      617 non-null    float64
 20  CH      617 non-null    float64
 21  CL      617 non-null    float64
 22  CR

In [10]:
train_data.isnull().sum()

Id        0
AB        0
AF        0
AH        0
AM        0
AR        0
AX        0
AY        0
AZ        0
BC        0
BD        0
BN        0
BP        0
BQ       60
BR        0
BZ        0
CB        2
CC        3
CD        0
CF        0
CH        0
CL        0
CR        0
CS        0
CU        0
CW        0
DA        0
DE        0
DF        0
DH        0
DI        0
DL        0
DN        0
DU        1
DV        0
DY        0
EB        0
EE        0
EG        0
EH        0
EJ        0
EL       60
EP        0
EU        0
FC        1
FD        0
FE        0
FI        0
FL        1
FR        0
FS        2
GB        0
GE        0
GF        0
GH        0
GI        0
GL        1
Class     0
dtype: int64

In [11]:
train_data.describe()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
count,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,...,616.000000,617.000000,615.000000,617.000000,617.000000,617.000000,617.000000,617.000000,616.000000,617.000000
mean,0.477149,3502.013221,118.624513,38.968552,10.128242,5.545576,0.060320,10.566447,8.053012,5350.388655,...,5.433199,3.533905,0.421501,20.724856,131.714987,14679.595398,31.489716,50.584437,8.530961,0.175041
std,0.468388,2300.322717,127.838950,69.728226,10.518877,2.551696,0.416817,4.350645,65.166943,3021.326641,...,11.496257,50.181948,1.305365,9.991907,144.181524,19352.959387,9.864239,36.266251,10.327010,0.380310
min,0.081187,192.593280,85.200147,3.177522,8.138688,0.699861,0.025578,3.396778,1.229900,1693.624320,...,0.173229,0.497060,0.067730,4.102182,72.611063,13.038894,9.432735,0.897628,0.001129,0.000000
25%,0.252107,2197.345480,85.200147,12.270314,8.138688,4.128294,0.025578,8.129580,1.229900,4155.702870,...,0.173229,0.497060,0.067730,14.036718,72.611063,2798.992584,25.034888,23.011684,0.124392,0.000000
50%,0.354659,3120.318960,85.200147,20.533110,8.138688,5.031912,0.025578,10.461320,1.229900,4997.960730,...,3.028141,1.131000,0.250601,18.771436,72.611063,7838.273610,30.608946,41.007968,0.337827,0.000000
75%,0.559763,4361.637390,113.739540,39.139886,8.138688,6.431634,0.036845,12.969516,5.081244,6035.885700,...,6.238814,1.512060,0.535067,25.608406,127.591671,19035.709240,36.863947,67.931664,21.978000,0.000000
max,6.161666,28688.187660,1910.123198,630.518230,178.943634,38.270880,10.315851,38.971568,1463.693448,53060.599240,...,137.932739,1244.227020,31.365763,135.781294,1497.351958,143790.071200,81.210825,191.194764,21.978000,1.000000


In [12]:
train_data.drop(['Id'], axis=1, inplace=True)

In [13]:
modes = train_data.mode().iloc[0]
modes

AB          0.260653
AF         192.59328
AH         85.200147
AM         630.51823
AR          8.138688
AX          0.699861
AY          0.025578
AZ          3.396778
BC            1.2299
BD        1693.62432
BN           20.4798
BP        175.592097
BQ        344.644105
BR         51.216883
BZ        257.432377
CB          12.49976
CC          0.463816
CD           23.3876
CF          0.510888
CH          0.025074
CL          1.050225
CR          0.069225
CS         13.784111
CU          1.467522
CW           7.03064
DA          39.03668
DE         183.06793
DF           0.23868
DH          0.371688
DI          60.23247
DL           10.3456
DN         28.510672
DU          0.005518
DV           1.74307
DY          0.804068
EB          4.926396
EE          0.286201
EG          185.5941
EH          0.003042
EJ                 B
EL        109.125159
EP         78.526968
EU          3.828384
FC         14.845488
FD           0.29685
FE       5088.922912
FI           3.58345
FL          0

In [14]:
train_data.fillna(modes, inplace=True)

In [15]:
train_data.isna().sum()

AB       0
AF       0
AH       0
AM       0
AR       0
AX       0
AY       0
AZ       0
BC       0
BD       0
BN       0
BP       0
BQ       0
BR       0
BZ       0
CB       0
CC       0
CD       0
CF       0
CH       0
CL       0
CR       0
CS       0
CU       0
CW       0
DA       0
DE       0
DF       0
DH       0
DI       0
DL       0
DN       0
DU       0
DV       0
DY       0
EB       0
EE       0
EG       0
EH       0
EJ       0
EL       0
EP       0
EU       0
FC       0
FD       0
FE       0
FI       0
FL       0
FR       0
FS       0
GB       0
GE       0
GF       0
GH       0
GI       0
GL       0
Class    0
dtype: int64

In [16]:
test_data.isna().sum()

Id     0
AB     0
AF     0
AH     0
AM     0
AR     0
AX     0
AY     0
AZ     0
BC     0
BD     0
BN     0
BP     0
BQ     0
BR     0
BZ     0
CB     0
CC     0
CD     0
CF     0
CH     0
CL     0
CR     0
CS     0
CU     0
CW     0
DA     0
DE     0
DF     0
DH     0
DI     0
DL     0
DN     0
DU     0
DV     0
DY     0
EB     0
EE     0
EG     0
EH     0
EJ     0
EL     0
EP     0
EU     0
FC     0
FD     0
FE     0
FI     0
FL     0
FR     0
FS     0
GB     0
GE     0
GF     0
GH     0
GI     0
GL     0
dtype: int64

In [17]:
train_data.describe(include='object')

,EJ
count,617
unique,2
top,B
freq,395


In [18]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train_data['EJ'] = encoder.fit_transform(train_data['EJ'])
test_data['EJ'] = encoder.transform(test_data['EJ'])

In [19]:
# Removing outliers:=
def removeOutliers(col):
  Q1 = train_data[col].quantile(0.25)
  Q3 = train_data[col].quantile(0.75)
  IQR = Q3-Q1
  lowerLimit = Q1-1.5*IQR
  upperLimit = Q3+1.5*IQR

  mean = train_data[((train_data[col] >= lowerLimit) | (train_data[col] <= upperLimit))][col].mean()
  train_data.loc[((train_data[col] < lowerLimit) | (train_data[col] > upperLimit)), col] = mean



In [20]:
colsFloat = train_data.select_dtypes('float64').columns
colsInt = train_data.select_dtypes('int64').columns

In [21]:
! pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.0 MB/s eta 0:00:00


In [22]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.4 MB/s eta 0:00:00


In [23]:
# Training models :
from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
import optuna

In [24]:
cols = [i for i in train_data.columns if i != 'Class']
seed = np.random.seed(6)
X = train_data[cols]
y = train_data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

In [48]:
# Random Forest :
params = {'max_depth': 15, 'n_estimators': 877, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': None}

rfmodel = RandomForestClassifier(**params,random_state=seed,criterion="log_loss")
rfmodel.fit(X_train,y_train)

RandomForestClassifier(criterion='log_loss', max_depth=15, max_features=None,
                       min_samples_split=3, n_estimators=877)

In [57]:
# XGBClassifier:
params = {
    'n_estimators': 2884,
        'max_depth': 20,
        'min_child_weight': 2.934487833919741,
        'learning_rate': 0.00133900801287575,
        'subsample': 0.9045063514419968,
        'gamma': 0.4329153382843715,
        'colsample_bytree': 0.38872702868412506,
        'colsample_bylevel': 0.8321880031718571,
        'colsample_bynode': 0.802355707802605
}

xgbmodel = XGBClassifier(**params, random_state=seed, eval_metric='logloss')
xgbmodel.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8321880031718571,
              colsample_bynode=0.802355707802605,
              colsample_bytree=0.38872702868412506, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.4329153382843715, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.00133900801287575,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=20, max_leaves=None,
              min_child_weight=2.934487833919741, missing=nan,
              monotone_constraints=None, n_estimators=2884, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

In [50]:
#lightGBM :
params = {
    'n_estimators': 91,
    'max_depth': 15,
    'learning_rate': 0.00144900801287575,
    'min_child_weight': 3.3909127244994792,
    'min_child_samples': 48,
    'subsample': 0.7432022782421412,
    'subsample_freq': 1,
    'colsample_bytree': 0.6522591366402789
}

lgbmmodel = LGBMClassifier(**params, random_state=seed)
lgbmmodel.fit(X_train, y_train)

LGBMClassifier(colsample_bytree=0.6522591366402789,
               learning_rate=0.00144900801287575, max_depth=15,
               min_child_samples=48, min_child_weight=3.3909127244994792,
               n_estimators=91, subsample=0.7432022782421412, subsample_freq=1)

# **Emsembling XGB, LGBM using Voting Classifier:**

In [58]:
wts = [random.uniform(0.8,1) for i in range(2)]
wts.extend([random.uniform(0.4,0.7) for i in range(1)])
print(wts)

[0.9230153665084343, 0.8612773546871542, 0.44805544716446416]


In [59]:
vcmodel = VotingClassifier([('lgbm', lgbmmodel), ('xgb', xgbmodel), ('rf', rfmodel)], voting="soft",weights=wts)
vcmodel.fit(X_train, y_train)

VotingClassifier(estimators=[('lgbm',
                              LGBMClassifier(colsample_bytree=0.6522591366402789,
                                             learning_rate=0.00144900801287575,
                                             max_depth=15, min_child_samples=48,
                                             min_child_weight=3.3909127244994792,
                                             n_estimators=91,
                                             subsample=0.7432022782421412,
                                             subsample_freq=1)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=0.832188003...
                                            min_child_weight=2.934487833919741,
                                            missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=2884, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None, ...)),
                             ('rf',
                              RandomForestClassifier(criterion='log_loss',
                                                     max_depth=15,
                                                     max_features=None,
                                                     min_samples_split=3,
                                                     n_estimators=877))],
                 voting='soft',
                 weights=[0.9230153665084343, 0.8612773546871542,
                          0.44805544716446416])

In [60]:
# Evaluating Models:

print("-"*100)
print("Accuracy of RF:",accuracy_score(y_test,rfmodel.predict(X_test)))
print("ROC AUC of RF:",roc_auc_score(y_test,rfmodel.predict_proba(X_test)[:,1]))
print("Cross Val Loss:",-1*np.mean(cross_val_score(rfmodel,X,y,cv=5,scoring='neg_log_loss')))

----------------------------------------------------------------------------------------------------
Accuracy of RF: 0.8709677419354839
ROC AUC of RF: 0.9451871657754012
Cross Val Loss: 0.22928951525004


In [61]:
print("-"*100)
print("Accuracy of LGBM:",accuracy_score(y_test,lgbmmodel.predict(X_test)))
print("ROC AUC of LGBM:",roc_auc_score(y_test,lgbmmodel.predict_proba(X_test)[:,1]))
print("Cross Val Loss:",-1*np.mean(cross_val_score(lgbmmodel,X,y,cv=5,scoring='neg_log_loss')))

----------------------------------------------------------------------------------------------------
Accuracy of LGBM: 0.8225806451612904
ROC AUC of LGBM: 0.8961675579322639
Cross Val Loss: 0.43162343848945406


In [62]:
print("-"*100)
print("Accuracy of XGB:",accuracy_score(y_test,xgbmodel.predict(X_test)))
print("ROC AUC of XGB:",roc_auc_score(y_test,xgbmodel.predict_proba(X_test)[:,1]))
print("Cross Val Loss:",-1*np.mean(cross_val_score(xgbmodel,X,y,cv=5,scoring='neg_log_loss')))

----------------------------------------------------------------------------------------------------
Accuracy of XGB: 0.9193548387096774
ROC AUC of XGB: 0.9656862745098038
Cross Val Loss: 0.2262556682526669


In [63]:
print("-"*100)
print("Accuracy of VC:",accuracy_score(y_test,vcmodel.predict(X_test)))
print("ROC AUC of VC:",roc_auc_score(y_test,vcmodel.predict_proba(X_test)[:,1]))
print("Cross Val Loss:",-1*np.mean(cross_val_score(vcmodel,X,y,cv=5,scoring='neg_log_loss')))


----------------------------------------------------------------------------------------------------
Accuracy of VC: 0.8790322580645161
ROC AUC of VC: 0.9581105169340464
Cross Val Loss: 0.28361970229235983
